In [ ]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import mrtrix3, utility as niu

In [ ]:
def init_mrtrix_wf():
    wf = pe.Workflow(name="mrtrix_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    denoise = pe.Node(mrtrix3.DWIDenoise(), name="denoise")
    
    unring = pe.Node(mrtrix3.MRDeGibbs(), name="unring")
    
    biascorr = pe.Node(mrtrix3.DWIBiasCorrect(use_fsl=True), name="biascorr")
    
    wf.connect([(inputnode, denoise, [("dwi_file", "in_file")]),
                (denoise, unring, [("out_file", "in_file")]),
                (unring, biascorr, [("out_file", "in_file")]),
                (inputnode, biascorr, [("bvec_file", "in_bvec"),
                                       ("bval_file", "in_bval")]),
                (biascorr, outputnode, [("out_file", "out_file")])
               ])
    
    return wf

In [ ]:
data_dir = os.path.abspath("../data")
layout = BIDSLayout(data_dir)

In [ ]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [ ]:
dwi_file = dwi_files[0]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)

In [ ]:
test_wf = init_mrtrix_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True

test_wf.run()